# Tutorial: Working with this stactools subpackage

Stactools ([docs](https://stactools.readthedocs.io/en/latest/), [source](https://github.com/stac-utils/stactools)) is a command line tool and library for working with [STAC](https://stacspec.org/), based on [PySTAC](https://github.com/stac-utils/pystac).

[Stactools dataset packages](https://github.com/stactools-packages) are add-ons for stactools that provide STAC functionality for specific datasets, such as [Sentinel 2](https://github.com/stactools-packages/sentinel2) and [Landsat](https://github.com/stactools-packages/landsat).

Stactools and its dataset packages can be accessed from the CLI or from within normal Python code. This notebook provides examples of both.

More information about the AAFC Landcover dataset can be found [here](https://open.canada.ca/data/en/dataset/18e3ef1a-497c-40c6-8326-aac1a34a0dec).

## 1. Using this notebook

The easiest way to use this notebook is to run it through `scripts/notebook`. This will create a Docker container Ready to run this notebook, which can be found in `docs/`.

If you wish to use this notebook outside of the container (such as on [mybinder.org](mybinder.org)) then please install the prerequisites using:

In [ ]:
!pip install stactools-aafc-landuse

## 2. From the CLI

The first thing we can do is check that the `stac` and `stac-aafc-landuse` CLI tools are installed and explore the options. Notice the inclusion of the `aafclanduse` command for `stac`:

In [1]:
!stac

Usage: stac [OPTIONS] COMMAND [ARGS]...

Options:
  -v, --verbose  Use verbose mode
  -q, --quiet    Use quiet mode (no output)
  --help         Show this message and exit.

Commands:
  aafclanduse  Commands for working with AAFC Land Use data
  copy         Copy a STAC Catalog
  describe     Prints out a list of all catalogs, collections and items in
               this STAC.

  info         Display info about a static STAC catalog.
  layout       Reformat the layout of a STAC based on templating.
  merge        Merge items from one STAC into another.
  move-assets  Move or copy assets in a STAC to the Item locations.
  validate     Validate a stac object.
  version      Display version info.


You can now explore the STAC dataset package commands to ingest and describe the data

In [2]:
!stac aafclanduse --help

Usage: stac aafclanduse [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  create-collection  Creates a STAC collection from AAFC Land Use metadata
  create-item        Create a STAC item from an AAFC Land Use tif


And more specific help with an individual command.

In [10]:
!stac aafclanduse create-collection --help

Usage: stac aafclanduse create-collection [OPTIONS]

  Creates a STAC Collection from AAFC Land Use metadata

  Args:     destination (str): Directory to create the collection json
  metadata (str): Path to a jsonld metadata file - provided by AAFC Returns:
  Callable

Options:
  -d, --destination TEXT  The output directory for the STAC Collection json
                          [required]

  -m, --metadata TEXT     The url to the metadata jsonld
  --help                  Show this message and exit.


In [11]:
!stac aafclanduse create-item --help

Usage: stac aafclanduse create-item [OPTIONS]

  Creates a STAC Item from an AAFC Land Use raster and accompanying metadata
  file.

  Args:     source (str): Path to an AAFC Land Use tif     destination
  (str): Directory where a COG and STAC item json will be created
  metadata (str): Path to a jsonld metadata file - provided by AAFC Returns:
  Callable

Options:
  -s, --source TEXT       Path to an AAFC Land Use tif  [required]
  -d, --destination TEXT  The output directory for the STAC json and COG
                          [required]

  -m, --metadata TEXT     The url to the metadata description.
  --help                  Show this message and exit.


The metadata necessary to create the root STAC collection is contained within the dataset package, so only an output directory is required:

In [9]:
!stac aafclanduse create-collection -d "."
!head ./aafc-landuse.json

{
  "type": "Collection",
  "id": "aafc-landuse",
  "stac_version": "1.0.0",
  "description": "The 1990, 2000 and 2010 Land Use (LU) maps cover all areas of Canada south of 60 degrees N at a spatial resolution of 30 metres. The LU classes follow the protocol of the Intergovernmental Panel on Climate Change (IPCC) and consist of: Forest, Water, Cropland, Grassland, Settlement and Otherland.",
  "links": [
    {
      "rel": "root",
      "href": "./aafc-landuse.json",
      "type": "application/json"


Creating a STAC item also requires the path to the corresponding zipped image path. This command will download, unzip and convert the `.tif` file to a cloud optimized geotiff (COG), then include a link to the COG in the item as an asset. You can explore the zipped images through the FTP [here](https://www.agr.gc.ca/atlas/data_donnees/lcv/aafcLand_Use/tif/).

In [13]:
# Create a STAC Item and COG - this can be done using remote or local .tif or .zip assets
!stac aafclanduse create-item -s "https://www.agr.gc.ca/atlas/data_donnees/lcv/aafcLand_Use/tif/2010/IMG_AAFC_LANDUSE_Z07_2010.zip" -d "."
!ls
!head IMG_AAFC_LANDUSE_Z07_2010.json

output: b'Input file size is 15252, 44639\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'
aafc-landuse.json		   IMG_AAFC_LANDUSE_Z07_2010_cog.tif.aux.xml
conf.py				   IMG_AAFC_LANDUSE_Z07_2010.json
IMG_AAFC_LANDUSE_Z07_2010_cog.tif  installation_and_basic_usage.ipynb
{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "IMG_AAFC_LANDUSE_Z07_2010",
  "properties": {
    "title": "IMG_AAFC_LANDUSE_Z07_2010",
    "description": "The 1990, 2000 and 2010 Land Use (LU) maps cover all areas of Canada south of 600N at a spatial resolution of 30 metres. The LU classes follow the protocol of the Intergovernmental Panel on Climate Change (IPCC) and consist of: Forest, Water, Cropland, Grassland, Settlement and Otherland.   \n  \n  The 1990, 2000 and 2010 Land Use (LU) maps were developed in response to a need for explicit, high-accuracy, high-resolution land use data to meet AAFC\u2019s commitments in international reporting, especially for the annual National Inventory Rep

## 3. As a Python module

So far we've used IPython [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to work with stactools packages on the command line from this notebook, but it's also possible to use them within Python scripts.

In [22]:
from stactools.aafc_landuse.constants import JSONLD_HREF
from stactools.aafc_landuse import utils, cog, stac
import os

Like above it's possible to create a collection from the metadata within the dataset package:

In [23]:
metadata = utils.get_metadata(JSONLD_HREF)
destination = "."
fname = "aafc-landuse.json"
collection = stac.create_collection(metadata, os.path.join(destination, fname))
collection.to_dict()

{'type': 'Collection',
 'id': 'aafc-landuse',
 'stac_version': '1.0.0',
 'description': 'The 1990, 2000 and 2010 Land Use (LU) maps cover all areas of Canada south of 60 degrees N at a spatial resolution of 30 metres. The LU classes follow the protocol of the Intergovernmental Panel on Climate Change (IPCC) and consist of: Forest, Water, Cropland, Grassland, Settlement and Otherland.',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': './aafc-landuse.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': 'license',
   'href': 'https://open.canada.ca/en/open-government-licence-canada',
   'title': 'Open Government Licence - Canada'},
  {'rel': <RelType.SELF: 'self'>,
   'href': '/home/jvrt/gdspark/202105_stactools/aafc-landuse/docs/aafc-landuse.json',
   'type': <MediaType.JSON: 'application/json'>}],
 'stac_extensions': [],
 'title': 'Land Use 1990, 2000 & 2010',
 'extent': {'spatial': {'bbox': [[-132.0, 36.0, -60.0, 60.0]]},
  'temporal': {'interval': [['1990-01-01T00:

And similarly to create an item:

In [25]:
source = "https://www.agr.gc.ca/atlas/data_donnees/lcv/aafcLand_Use/tif/2010/IMG_AAFC_LANDUSE_Z07_2010.zip"
destination = "."

# Access/download src tif and create a COG
with utils.AssetManager(source) as asset:
    asset_tif = asset.path
    cog_path = os.path.join(
        destination,
        os.path.splitext(os.path.basename(asset_tif))[0] + "_cog.tif",
    )
    cog.create_cog(asset_tif, cog_path, dry_run=False)

# Create stac item
json_path = cog_path[:-8] + ".json"
item = stac.create_item(metadata, json_path, cog_path)
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'IMG_AAFC_LANDUSE_Z07_2010',
 'properties': {'title': 'IMG_AAFC_LANDUSE_Z07_2010',
  'description': 'The 1990, 2000 and 2010 Land Use (LU) maps cover all areas of Canada south of 600N at a spatial resolution of 30 metres. The LU classes follow the protocol of the Intergovernmental Panel on Climate Change (IPCC) and consist of: Forest, Water, Cropland, Grassland, Settlement and Otherland.   \n  \n  The 1990, 2000 and 2010 Land Use (LU) maps were developed in response to a need for explicit, high-accuracy, high-resolution land use data to meet AAFC’s commitments in international reporting, especially for the annual National Inventory Report (NIR) to the United Nations Framework Convention on Climate Change (UNFCCC), the Agri-Environmental program of the Organization for Economic Co-operation and Development (OECD) and the FAOSTAT component of the Food and Agriculture Organization of the United Nations (FAO).',
  'start_datetime': '2010